In [22]:

import pickle
import pandas as pd

from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service as ChromeService

from selenium.webdriver.firefox.service import Service as FirefoxService
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.firefox.options import Options


from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from selenium_stealth import stealth
import random
import time
import json
import unidecode


text file to dataframe

In [23]:
def read_list_drives(file_path):
    list_drive = []
    with open(file_path, "r") as file:
        for line in file:
            # Parse the JSON data
            drive_data = json.loads(line.strip())
            list_drive.append(drive_data)
    return list_drive

Scrap lien drive

In [24]:
def scrap_lien_drive(driver,list_lien_drives =[]):

    driver.get("https://www.intermarche.com/enseigne/magazine/tous-les-magasins")
   
    time.sleep(2)
    
    buttonDecline = WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'didomi-continue-without-agreeing'))
        )
    buttonDecline.click()
    time.sleep(2)
    regionTag= driver.find_elements(By.CLASS_NAME, 'storeInfoList__section')
    for region in regionTag:
        departmentTag = region.find_elements(By.CLASS_NAME, 'storeInfoList__department')
        for department in departmentTag:
            try :
                buttonPlus = department.find_element(By.CLASS_NAME, 'storeInfoList__button')
                buttonPlus.click()
            except :
                pass
            driveTag = department.find_elements(By.CLASS_NAME, 'storeInfoList__store')
            for drive in driveTag:
                lienDrive = drive.find_element(By.TAG_NAME, 'a').get_attribute("href")
                list_lien_drives.append(lienDrive)
                print(lienDrive)
    

    # Open the file in write mode and write the contents of the list to it
    with open('list_lien_drives.txt', "w") as file:
        for item in list_lien_drives:
            file.write("%s\n" % item)

    

    
    return(list_lien_drives)

Scrap Drive

In [25]:
def scrapDrive(driver):

    with open('list_lien_drives.txt', "r") as file:
        list_lien_drives = file.readlines()

    # Read the last processed index from the file
    try:
        with open('last_processed_index.txt', "r") as index_file:
            last_processed_index = int(index_file.read().strip())
    except FileNotFoundError:
        last_processed_index = 0

    # Resume from the last processed index
    list_lien_drives = list_lien_drives[last_processed_index:]

    for i, lien in enumerate(list_lien_drives):
    # Remove newline characters from the URL
        lien = lien.strip()
        driver.get(lien)
        
        nomDrive = WebDriverWait(driver, 300).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'pdvHeading__title'))
        ).text
        
        adresse = driver.find_element(By.CLASS_NAME, 'addressAndMap__address__streets').text
        
        coordScript = driver.find_element(By.XPATH, '//*[@id="__NEXT_DATA__"]')
        
        # Extract the text content of the script block
        script_text = coordScript.get_attribute("textContent")

        # Parse the JSON data
        data = json.loads(script_text)

        # Access the latitude and longitude values
        latitude = data['props']['pageProps']['pdv']['address']['latitude']
        longitude = data['props']['pageProps']['pdv']['address']['longitude']
        dict_drive ={
                "nom_drive": nomDrive,
                'adresse': adresse,
                "lien_drive": lien,
                'latitude':latitude,
                'longitude':longitude
            }
        print(dict_drive)
       

        

        with open('list_drives.txt', "a") as file:
            file.write(json.dumps(dict_drive) + "\n")
            
        # Save the index of the last processed URL to a file
        with open('last_processed_index.txt', "w") as index_file:
            index_file.write(str(last_processed_index + i + 1))
        #time.sleep(1.5)
  
    list_drive = read_list_drives("list_drives.txt")
    
    df_drive = pd.DataFrame(list_drive)
    df_drive.to_csv("list_drives_intermarcheBrut.csv")
    
    return(df_drive)

find last index of a specific character in string 

In [26]:
def find_last_index(string, char, indices = []):
    for i in range(len(string)):
        if string[i] == char:
            indices.append(i)

    last_indice = int(indices[-1])
    return(last_indice)

Process Drives


In [27]:
def process_drive(df,list_drives = []):
    for i,adresse in enumerate(df["adresse"]):

        saut = find_last_index(string = adresse, char='\n')

        department = adresse[saut+1:saut +3]
        city = adresse[saut+1:]
        city = city.replace(",","") 
        
        reservedCharacter = ["!","*","'","(",")",";",":","@","&","=","+","$",",","/","?","%","#","[","]"," "]
        nom_drive = df.iloc[i]['nom_drive'].replace('\n',"")
        nom_driveUrl = nom_drive
        for character in reservedCharacter:
            nom_driveUrl = nom_driveUrl.replace(character,"_")
        nom_driveUrl = unidecode.unidecode(nom_driveUrl)

        dict_drive ={
            "supermarket":"Intermarche",
            "department": department,
            "nom_drive": nom_drive,
            "nom_driveUrl": nom_driveUrl,
            'city': city,
            "adresse": adresse.replace('\n',""),
            "latitude": df.iloc[i]['latitude'],
            "longitude": df.iloc[i]['longitude'],     
            "lien_drive" : df.iloc[i]['lien_drive']

        }
        list_drives.append(dict_drive)
        print(dict_drive)
    df_drives = pd.DataFrame(list_drives)
    
    df_drives.to_csv("list_drives_intermarche.csv")

Main

In [28]:
#set up driver
# create a new Service instance and specify path to Chromedriver executable
service = ChromeService(executable_path=ChromeDriverManager().install())


# Step 2: Change browser properties
# create a ChromeOptions object
options = webdriver.ChromeOptions()

#run in headless mode
#options.add_argument("--headless")

# disable the AutomationControlled feature of Blink rendering engine
options.add_argument('--disable-blink-features=AutomationControlled')
 
# disable pop-up blocking
options.add_argument('--disable-popup-blocking')
 
# start the browser window in maximized mode
options.add_argument('--start-maximized')
 
 
# disable sandbox mode
options.add_argument('--no-sandbox')
 
# disable shared memory usage
options.add_argument('--disable-dev-shm-usage')
options.add_extension('../../Solver-CAPTCHA-gratuit-auto-hCAPTCHA-reCAPTCHA.crx')
options.add_extension('../../Buster-Captcha-Solver-for-Humans.crx')
options.add_extension('../../Get-cookies-txt-LOCALLY.crx')

# Set navigator.webdriver to undefined
# create a driver instance
driver = webdriver.Chrome(service=service, options=options)

# Change the property value of the navigator for webdriver to undefined
driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")



# Step 3: Rotate user agents 
user_agents = [
    # Add your list of user agents here
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 13_1) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15',
]

# select random user agent
user_agent = random.choice(user_agents)

# pass in selected user agent as an argument
options.add_argument(f'user-agent={user_agent}')

# set user agent using execute_cpd_cmd
driver.execute_cdp_cmd('Network.setUserAgentOverride', {"userAgent": user_agent})


# Step 4: Scrape using Stealth
#enable stealth mode
stealth(driver,
        languages=["en-US", "en"],
        vendor="Google Inc.",
        platform="Win32",
        webgl_vendor="Intel Inc.",
        renderer="Intel Iris OpenGL Engine",
        fix_hairline=True,
        )

#scrapDrive(driver)
df = pd.read_csv('list_drives_intermarcheBrut.csv')
process_drive(df)


{'supermarket': 'Intermarche', 'department': '01', 'nom_drive': 'Hyper Amberieu en Bugey', 'nom_driveUrl': 'Hyper_Amberieu_en_Bugey', 'city': '01500 Amberieu en Bugey', 'adresse': "Centre commercial de l'aviation-Les terreaux-01500 Amberieu en Bugey", 'latitude': 45.971973, 'longitude': 5.337993, 'lien_drive': 'https://www.intermarche.com/magasins/10498/amberieu-en-bugey-01500/infos-pratiques'}
{'supermarket': 'Intermarche', 'department': '01', 'nom_drive': 'Super Béon', 'nom_driveUrl': 'Super_Beon', 'city': '01350 Béon', 'adresse': '2 Rue des Roselières-01350 Béon', 'latitude': 45.847511, 'longitude': 5.767197, 'lien_drive': 'https://www.intermarche.com/magasins/05667/beon-01350/infos-pratiques'}
{'supermarket': 'Intermarche', 'department': '01', 'nom_drive': 'Super Bourg en Bresse', 'nom_driveUrl': 'Super_Bourg_en_Bresse', 'city': '01000 Bourg en Bresse', 'adresse': '56 rue du Stand-01000 Bourg en Bresse', 'latitude': 46.19536, 'longitude': 5.229369, 'lien_drive': 'https://www.interm